In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Capstone/traindata.csv')

In [ ]:
df.head(3)

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,7,20,Friday,7.192192e+09,1,FROZEN FOODS,9117.0
1,7,20,Friday,6.811311e+10,2,SERVICE DELI,4010.0
2,25,28,Friday,8.805520e+11,1,LADIESWEAR,313.0


In [ ]:
df.groupby('VisitNumber')['Weekday'].nunique()

VisitNumber
20        1
28        1
43        1
72        1
76        1
         ..
191318    1
191322    1
191326    1
191337    1
191343    1
Name: Weekday, Length: 12309, dtype: int64

In [ ]:
weekday_mapping = {'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4, 'Friday':5,\
                   'Saturday':6, 'Sunday':7}
df['Weekday'] = df['Weekday'].apply(lambda x: weekday_mapping[x])
df.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,7,20,5,7.192192e+09,1,FROZEN FOODS,9117.0
1,7,20,5,6.811311e+10,2,SERVICE DELI,4010.0
2,25,28,5,8.805520e+11,1,LADIESWEAR,313.0
3,25,28,5,8.085947e+10,1,LADIESWEAR,4447.0
4,25,28,5,4.900004e+09,1,DSD GROCERY,9538.0


In [ ]:
x1 = pd.DataFrame(df.groupby('VisitNumber')['Weekday'].unique())
x1.head(3)

,Weekday
VisitNumber,
20,[5]
28,[5]
43,[5]


In [ ]:
'[5]'.split('[')[1][0]

'5'

In [ ]:
x1['Weekday']= x1['Weekday'].apply(lambda x: str(x).split('[')[1][0])
x1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12309 entries, 20 to 191343
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Weekday  12309 non-null  object
dtypes: object(1)
memory usage: 192.3+ KB


In [ ]:
x1['Weekday']= x1['Weekday'].astype('int32')

In [ ]:
x1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12309 entries, 20 to 191343
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Weekday  12309 non-null  int32
dtypes: int32(1)
memory usage: 144.2 KB


In [ ]:
def find_weekend(x):
  if(x<6):
    return 0
  return 1

In [ ]:
x1['Weekend']= x1['Weekday'].apply(lambda x: find_weekend(x))

In [ ]:
x1.head(3)

,Weekday,Weekend
VisitNumber,,
20,5,0
28,5,0
43,5,0


In [ ]:
x2 = pd.DataFrame(df.groupby('VisitNumber')['Weekday'].count()).reset_index()
x2.columns = ['VisitNumber', 'Visitcnt']
x2.head(3)

,VisitNumber,Visitcnt
0,20,2
1,28,8
2,43,4


In [ ]:
x1.shape, x2.shape

((12309, 2), (12309, 2))

In [ ]:
df['Buy']= np.where(df['ScanCount']>0,df['ScanCount'],0)
df['Sell']= np.where(df['ScanCount']<0,df['ScanCount'],0)

In [ ]:
df.head(3)

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Buy,Sell
0,7,20,5,7.192192e+09,1,FROZEN FOODS,9117.0,1,0
1,7,20,5,6.811311e+10,2,SERVICE DELI,4010.0,2,0
2,25,28,5,8.805520e+11,1,LADIESWEAR,313.0,1,0


In [ ]:
x3 = pd.DataFrame(df.groupby('VisitNumber').agg(NetBuy= ('ScanCount','sum'),\
                                                         TotalBuy=('Buy','sum'),\
                                                         Totalreturn=('Sell','sum'))).reset_index()
x3.head(3)

,VisitNumber,NetBuy,TotalBuy,Totalreturn
0,20,3,3,0
1,28,8,8,0
2,43,4,4,0


In [ ]:
x4 = pd.DataFrame(df.groupby('VisitNumber')['DepartmentDescription'].nunique()).reset_index()
x4.columns = ['VisitNumber', 'UniqueDepts']
x4.head(3)

,VisitNumber,UniqueDepts
0,20,2
1,28,4
2,43,4


In [ ]:
df['DepartmentDescription'].nunique()

64

In [ ]:
dd_df= pd.get_dummies(df['DepartmentDescription'])

In [ ]:
df.shape, dd_df.shape

((77186, 9), (77186, 64))

In [ ]:
dd_df= pd.concat([df['VisitNumber'],dd_df], axis=1)

In [ ]:
dd_df.head(3)

,VisitNumber,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,28,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
x5 = dd_df.groupby('VisitNumber').sum()
x5.head(3)

,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
VisitNumber,,,,,,,,,,,,,,,,,,,,,
20,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
28,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
x1.shape, x2.shape, x3.shape, x4.shape, x5.shape

((12309, 2), (12309, 2), (12309, 4), (12309, 2), (12309, 64))

In [ ]:
x_ft = x1.merge(x2, on='VisitNumber', how='inner')
x_ft.head(3)

,VisitNumber,Weekday,Weekend,Visitcnt
0,20,5,0,2
1,28,5,0,8
2,43,5,0,4


In [ ]:
x_ft = x_ft.merge(x3, on='VisitNumber', how='inner')
x_ft = x_ft.merge(x4, on='VisitNumber', how='inner')
x_ft.head(3)

,VisitNumber,Weekday,Weekend,Visitcnt,NetBuy,TotalBuy,Totalreturn,UniqueDepts
0,20,5,0,2,3,3,0,2
1,28,5,0,8,8,8,0,4
2,43,5,0,4,4,4,0,4


In [ ]:
x_ft = x_ft.merge(x5, on='VisitNumber', how='inner')
x_ft.head(3)

,VisitNumber,Weekday,Weekend,Visitcnt,NetBuy,TotalBuy,Totalreturn,UniqueDepts,1-HR PHOTO,ACCESSORIES,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,20,5,0,2,3,3,0,2,0,0,...,0,0,1,0,0,0,0,0,0,0
1,28,5,0,8,8,8,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
2,43,5,0,4,4,4,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
trips = pd.DataFrame(df.groupby(['VisitNumber', 'TripType'])['Weekday'].count()).reset_index()
trips = trips[['VisitNumber', 'TripType']]
trips.head(3)

,VisitNumber,TripType
0,20,7
1,28,25
2,43,38


In [ ]:
final_df = x_ft.merge(trips, how= 'inner', on='VisitNumber')
final_df.head(3)

,VisitNumber,Weekday,Weekend,Visitcnt,NetBuy,TotalBuy,Totalreturn,UniqueDepts,1-HR PHOTO,ACCESSORIES,...,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS,TripType
0,20,5,0,2,3,3,0,2,0,0,...,0,1,0,0,0,0,0,0,0,7
1,28,5,0,8,8,8,0,4,0,0,...,0,0,0,0,0,0,0,0,0,25
2,43,5,0,4,4,4,0,4,0,0,...,0,0,0,0,0,0,0,0,0,38


In [ ]:
X_features = final_df.iloc[:,1:-1]
Y = final_df['TripType']
X_features.shape, Y.shape

((12309, 71), (12309,))

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
ss= StandardScaler()
X= pd.DataFrame(ss.fit_transform(X_features))
X.shape

(12309, 71)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, stratify=Y, random_state= 78, test_size=0.20)
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

((9847, 71), (2462, 71), (9847,), (2462,))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
dt= DecisionTreeClassifier()
dt.fit(xtrain,ytrain)
pred_dt= dt.predict(xtest)
pred_dt[:5]

array([ 7, 38, 38, 25,  7])

In [ ]:
rf= RandomForestClassifier()
rf.fit(xtrain,ytrain)
pred_rf= rf.predict(xtest)
pred_rf[:5]

array([ 7, 38, 38, 25,  7])

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
accuracy_score(ytest, pred_dt), accuracy_score(ytest, pred_rf)

(0.904549147034931, 0.9260763606823721)

In [ ]:
print(classification_report(ytest, pred_dt))

              precision    recall  f1-score   support

           7       0.90      0.92      0.91      1140
          25       0.97      0.97      0.97       740
          38       0.84      0.79      0.82       582

    accuracy                           0.90      2462
   macro avg       0.90      0.89      0.90      2462
weighted avg       0.90      0.90      0.90      2462



In [ ]:
print(classification_report(ytest, pred_rf))

              precision    recall  f1-score   support

           7       0.92      0.94      0.93      1140
          25       0.97      0.98      0.97       740
          38       0.88      0.83      0.86       582

    accuracy                           0.93      2462
   macro avg       0.92      0.92      0.92      2462
weighted avg       0.93      0.93      0.93      2462



In [ ]:
import pickle

In [ ]:
with open('RF_model_reg.pkl', 'wb') as file:
  pickle.dump(rf, file)

In [ ]:
with open('Reg_ss.pkl', 'wb') as file:
  pickle.dump(ss,file)